<a href="https://colab.research.google.com/github/Hrudayangam/amazon_hackathon/blob/main/Product_Browse_Node_Classification_Active_Reality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import re
import nltk
from nltk.corpus import stopwords, words
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('words')
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical

#Defining functions for preprocessing of string type data 

In [ ]:
def Clean_a_String(string):  

  clean_string_list = re.sub('[^a-zA-Z]', ' ', string)
  #print(clean_string_list)
  clean_string_list = clean_string_list.lower()
  #print(clean_string_list)
  clean_string_list = clean_string_list.split()
  #print(clean_string_list)

  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')

  clean_string_list = [ps.stem(word) for word in clean_string_list if not word in set(all_stopwords)]
  clean_string_list = ' '.join(clean_string_list)
  """
  for i in clean_string_list:
    if i in words.words():
      continue
    else:
      clean_string_list.remove(i)
  """ 
  return clean_string_list

In [ ]:
def Clean_the_data_TITLE(dataset):
  List = []
  try:  
    for index in range(len(dataset)):
      string = dataset['TITLE'][index]
      if string != string:
        temp_list = ["NOT_REQUIRED"]

      else:
        temp_list = str(Clean_a_String(string))
        #temp_list += " " + str(dataset['BRAND'][index])
      
      List.append(temp_list)
      
  except TypeError:
    print(dataset['TITLE'][index])
  
  except KeyError:
    print(dataset['TITLE'][index])
  
  return List

In [ ]:
def Clean_the_data_OTHER(dataset): 
  List = []
  dataset['DESCRIPTION'] = dataset['DESCRIPTION'].fillna("0")
  dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].fillna("0")

  try:  
    for index in range(len(dataset)):
      string = dataset['DESCRIPTION'][index] + dataset['BULLET_POINTS'][index]
      if string != string:
        temp_list = []

      else:
        temp_list = Clean_a_String(string)
      
      List.append(temp_list)

  except TypeError:
    print(TypeError)
    print(dataset['TITLE'][index])
  
  except KeyError:
    print(KeyError)
    print(dataset['TITLE'][index])
  
  return List

#Data extraction

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Amazon ML/train.csv", escapechar="\\", quoting= csv.QUOTE_NONE)

#Data Pre-processing

In [ ]:
X = Clean_the_data_TITLE(data)
#X = Clean_the_data_OTHER(data)

*X will take values and x will produce combine 'TITLE' and 'BRAND' data for bag of words*

In [ ]:
x = []
for i in range(len(X)):
  temp = str(X[i]) + " " + str(data['BRAND'][i])
  #temp = str(X[i])
  x.append(temp)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
x=cv.fit_transform(x).toarray()

**Encoding for Y**

In [ ]:
y = data.iloc[:, -1].values
y = array(y)
print(y)
# one hot encode
y = to_categorical(y)

In [ ]:
y.shape

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2, random_state=42)

#**ANN**

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=1000, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1000, activation='relu'))
ann.add(tf.keras.layers.Dense(units=y.shape[1], activation='softmax'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(x_train, y_train, batch_size = 64, epochs = 15)

#ANN PREDICTIONS

In [ ]:
y_pred = ann.predict(x_test)

#Converting probability into Binary output in each list

In [ ]:
y_pred = np.around(y_pred)

#**ACCURACY**

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))